In [6]:
import pandas as pd
import numpy as np
import torch
import pytorch_lightning as pl
import mlflow
from mlflow.models import infer_signature
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from torch.utils.data import Dataset,random_split,DataLoader
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping,ModelCheckpoint
from pytorch_lightning.loggers import MLFlowLogger
from torchmetrics import MeanSquaredError

In [7]:
mlflow.set_tracking_uri("http://localhost:5000")

In [8]:
data=pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",header=0,delimiter=';')

In [9]:
train_x = data.drop('quality',axis=1).astype(np.float32)
train_y = data['quality'].astype(np.float32)
signature = infer_signature(train_x,train_y)

In [10]:
class QualityDataset(Dataset) :
    def __init__(self) :
        super().__init__()
        self.data =  pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
                            header=0,delimiter=';')
        self.label=self.data['quality'].values
        self.data=self.data.drop('quality',axis=1).values

    def __len__(self) :
        return len(self.data)
    
    def __getitem__(self,idx):
        x = torch.tensor(self.data[idx],dtype=torch.float32)
        y = torch.tensor(self.label[idx],dtype=torch.float32)
        return x,y

        


class QualityDataModule(pl.LightningDataModule) :

    def __init__(self,batch_size = 64) :
        super().__init__()
        self.batch_size = batch_size
    
    def prepare_data(self):
        pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
                            header=0,delimiter=';')
    
    def setup(self,stage=None) :
        generator = torch.Generator().manual_seed(42)
        self.train,self.test=random_split(QualityDataset(),[0.75,0.25],generator=generator)
        if stage == 'fit' :
          self.train,self.val = random_split(self.train,[0.8,0.2],generator=generator)
        
    def train_dataloader(self):
        return DataLoader(self.train,batch_size=self.batch_size)
    
    def val_dataloader(self):
        return DataLoader(self.val,batch_size=self.batch_size)
    
    def test_dataloader(self):
        return DataLoader(self.test,batch_size=self.batch_size)
    
           

In [14]:
class QualityModel(pl.LightningModule) :
    def __init__(self,learning_rate,momentum,in_features=11) :
        super().__init__()
        self.save_hyperparameters()
        self.in_features=in_features
        self.lr = learning_rate
        self.momentum = momentum
        self.model = torch.nn.Sequential(torch.nn.BatchNorm1d(num_features=in_features),torch.nn.Linear(in_features=in_features,out_features=64),
                                torch.nn.ReLU(),torch.nn.Dropout(p=0.2),torch.nn.Linear(in_features=64,out_features=32),
                                torch.nn.ReLU(),torch.nn.Linear(in_features=32,out_features=1))
        self.loss_fn = torch.nn.MSELoss()
        self.train_mse = MeanSquaredError()
        self.train_rmse = MeanSquaredError(squared=False)
        self.val_mse = MeanSquaredError()
        self.val_rmse = MeanSquaredError(squared=False)
    
    def forward(self,inputs) :
        return self.model(inputs)
    
    def training_step(self,batch,batch_idx):
        train,target = batch
        out = self(train)
        out = out.squeeze()
        train_loss = self.loss_fn(out,target)
        self.train_mse(out,target)
        self.train_rmse(out,target)
        self.log('train_loss',self.train_mse)
        self.log('train_rmse',self.train_rmse)
        
        return train_loss
    
    def validation_step(self,batch,batch_idx):
        val,target = batch
        out = self(val)
        out = out.squeeze()
        self.val_mse(out,target)
        self.val_rmse(out,target)
        self.log('val_loss',self.val_mse)
        self.log('val_rmse',self.val_rmse)

    
    def configure_optimizers(self):
        return torch.optim.SGD(params=self.parameters(),lr=self.lr,momentum=self.momentum)


In [15]:
def create_and_train_model(learning_rate, momentum,run_id, in_features=11,epochs=15,batch_size=64,experiment_name='quality-optimization_6'):
    dataloader = QualityDataModule(batch_size)
    model = QualityModel(learning_rate=learning_rate,momentum=momentum,in_features=in_features)
    early_stopping = EarlyStopping(monitor='val_loss',patience=3,mode='min')
    checkpoint = ModelCheckpoint(monitor='val_loss',mode='min',save_top_k=1,filename='{epoch}-{val_loss:.2f}-{val_rmse:.2f}')
    MLlogger = MLFlowLogger(experiment_name=experiment_name,run_id=run_id,tracking_uri="http://localhost:5000")
    trainer = Trainer(max_epochs=epochs,callbacks=[early_stopping,checkpoint],logger=MLlogger)
    trainer.fit(model,dataloader)

    return checkpoint



In [16]:
def objective(params) :

    with mlflow.start_run(nested=True) as run:
            run_id_for_this_trial = mlflow.active_run()
            checkpoint = create_and_train_model(params['learning_rate'],params['momentum'],run_id=run_id_for_this_trial.info.run_id)
            ckpt_path = checkpoint.best_model_path
            checkpoint_model = QualityModel.load_from_checkpoint(ckpt_path)
            mlflow.log_params(
            {
                "learning_rate": params["learning_rate"],
                "momentum": params["momentum"],
                "optimizer": "SGD",
                "architecture": "64-32-1",
            }
            )

            mlflow.pytorch.log_model(checkpoint_model, name="model",signature=signature)

    return {"loss": checkpoint.best_model_score.item(), "status": STATUS_OK ,'run_id' : run_id_for_this_trial.info.run_id}

In [17]:
search_space = {
    "learning_rate": hp.loguniform("learning_rate", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 0.9),
}
# Create or set experiment
experiment_name = "quality-optimization_6"
mlflow.set_experiment(experiment_name)

print(f"Starting hyperparameter optimization experiment: {experiment_name}")
print("This will run 15 trials to find optimal hyperparameters...")

with mlflow.start_run(run_name="hyperparameter-sweep"):
    # Log experiment metadata
    mlflow.log_params(
        {
            "optimization_method": "Tree-structured Parzen Estimator (TPE)",
            "max_evaluations": 15,
            "objective_metric": "validation_rmse",
            "dataset": "wine-quality",
            "model_type": "neural_network",
        }
    )

    # Run optimization
    trials = Trials()
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=15,
        trials=trials,
        verbose=True,
    )

    # Find and log best results
    best_trial = min(trials.results, key=lambda x: x["loss"])
    best_rmse = best_trial["loss"]

    # Log optimization results
    mlflow.log_params(
        {
            "best_learning_rate": best_params["learning_rate"],
            "best_momentum": best_params["momentum"],
        }
    )
    mlflow.log_metrics(
        {
            "best_val_rmse": best_rmse,
            "total_trials": len(trials.trials),
            "optimization_completed": 1,
        }
    )

Starting hyperparameter optimization experiment: quality-optimization_6
This will run 15 trials to find optimal hyperparameters...
  0%|          | 0/15 [00:00<?, ?trial/s, best loss=?]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



  | Name       | Type             | Params | Mode 
--------------------------------------------------------
0 | model      | Sequential       | 2.9 K  | train
1 | loss_fn    | MSELoss          | 0      | train
2 | train_mse  | MeanSquaredError | 0      | train
3 | train_rmse | MeanSquaredError | 0      | train
4 | val_mse    | MeanSquaredError | 0      | train
5 | val_rmse   | MeanSquaredError | 0      | train
--------------------------------------------------------
2.9 K     Trainable params
0         Non-trainable params
2.9 K     Total params
0.012     Total estimated model params size (MB)
13        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]    
  0%|          | 0/15 [00:01<?, ?trial/s, best loss=?]

/usr/local/python/3.12.1/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (46) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.



...

Training: |          | 0/? [00:00<?, ?it/s]                                      
Epoch 0: 100%|##########| 46/46 [00:00<00:00, 116.24it/s, v_num=64cd]            
Validation: |          | 0/? [00:00<?, ?it/s]                                    
Epoch 1: 100%|##########| 46/46 [00:00<00:00, 101.70it/s, v_num=64cd]            
Validation: |          | 0/? [00:00<?, ?it/s]                                    
Epoch 2: 100%|##########| 46/46 [00:00<00:00, 89.65it/s, v_num=64cd]             
Validation: |          | 0/? [00:00<?, ?it/s]                                    
Epoch 3: 100%|##########| 46/46 [00:00<00:00, 91.52it/s, v_num=64cd]             
Validation: |          | 0/? [00:00<?, ?it/s]                                    
Epoch 4: 100%|##########| 46/46 [00:00<00:00, 95.07it/s, v_num=64cd]             
Validation: |          | 0/? [00:00<?, ?it/s]                                    
Epoch 5: 100%|##########| 46/46 [00:00<00:00, 97.08it/s, v_num=64cd]             
Validation: |   

`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: 100%|##########| 46/46 [00:00<00:00, 58.81it/s, v_num=64cd]            
🏃 View run serious-hawk-668 at: http://localhost:5000/#/experiments/742523656056265699/runs/8aa61e7d11f141a3a9c1a9682f4364cd

🧪 View experiment at: http://localhost:5000/#/experiments/742523656056265699    

 93%|█████████▎| 14/15 [04:21<00:16, 16.08s/trial, best loss: 0.5232229828834534]

2025/08/18 17:16:37 WARNING mlflow.utils.requirements_utils: Found torch version (2.7.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.7.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.



🏃 View run serious-hawk-668 at: http://localhost:5000/#/experiments/742523656056265699/runs/8aa61e7d11f141a3a9c1a9682f4364cd

🧪 View experiment at: http://localhost:5000/#/experiments/742523656056265699    

100%|██████████| 15/15 [04:29<00:00, 17.95s/trial, best loss: 0.5232229828834534]
🏃 View run hyperparameter-sweep at: http://localhost:5000/#/experiments/742523656056265699/runs/ed006d42792f4e10ba4ecc452cc95a31
🧪 View experiment at: http://localhost:5000/#/experiments/742523656056265699
